__Demultiplexing the two sequencing libraries using barcode splitter and then trimmatic(?). Following Michelle's demultiplexing steps on https://github.com/pinskylab/genomics/blob/df5c254a0e097da18f49532f748a5a2ae5e1299f/Protocols/01_hiseq_workflow.md as a rough guide, and barcode_splitter.py documentation: https://pypi.org/project/barcode-splitter/__

__Start with sequencing library 1__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/


__Each capture (1-16) was a PCR pool, first separate out by PCR pool. First, in SP1__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/


__Find which files the barcodes are in, use ATCACG as the example barcode__

In [ ]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz | grep -c "^ATCACG"

In [ ]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | grep -c "^ATCACG"

__Look at the head of the compressed file to see where the barcode position is relative to index__

In [ ]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | grep "^ATCACG" | head

In [ ]:
!gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz | grep "^CATCGA" | head #reverse compliment of the read one barcode

__Okay it looks like the barcode is to the right of the index, left of the read as expected for the forward and reverse reads, and in reverse read the sequence is reversed__

__Where is the PCR index? Should be beginning of index read, use ATTACTCG as example__

In [ ]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz | grep "^ATTACTCG" | head

In [ ]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | head

In [ ]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz | head

__Count the number of reads__

In [ ]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | wc -l | awk '{print$1/4}'
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz  | wc -l | awk '{print$1/4}'

__Run barcode splitter in a screen on annotate__

In [ ]:
#run within screen annotate
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP1.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz --idxread 2  

__Now for SP2__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit


In [ ]:
#run within screen annotate or make sure this notebook is in a screen
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP2.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_4_passed_filter.fastq.gz --idxread 2  

__Now split out by barcoded adapter sequence, remove adapters, and trime low quality reads with ultraplex https://github.com/ulelab/ultraplex__

In [ ]:
#move to subdirectory
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/


In [ ]:
!#January 25, there's still barcodes in the beginning of the read 2 files after fastp adapter trimming. I'm going to try ultraplex instead.
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/


In [ ]:
#not sure why doing this in a bash for loop fails, but it's not worth figuring it out just run each capture pool separately
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP1_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP1 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP2-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP2-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP2_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP2 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP3-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP3-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP3_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP3 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP4-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP4-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP4_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP4 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP5-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP5-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP5_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP5 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP6-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP6-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP6_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP6 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP7-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP7-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP7_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP7 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP8-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP8-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP8_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP8 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP9-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP9-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP9_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP9 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP10-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP10-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP10_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP10 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP11-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP11-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP11_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP11 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP12-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP12-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP12_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP12 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP13-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP13-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP13_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP13 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP14-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP14-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP14_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP14 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP15-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP15-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP15_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP15 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP16_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP16 
#failed ultraplex bash loop
#for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/cap_filenames);      do ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/$i_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out -o $i;      don

In [ ]:
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP16_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP16 


__rename ultraplex output files__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out
!rename 'ultraplex_' '' *.gz #delete the ultraplex prefix
!rename 'Fwd' 'R1' *.gz #rename to read 1
!rename 'Rev' 'R2' *.gz #rename to read 2

__copy the de-multiplexed data to a fresh directory for fastq screen__

In [12]:
#!mkdir ~/ClownfishGWAS/data/APPC_sequencing/fastq_screen
!cp ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/$(ls *.gz -I '*no_match*') /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen #only move the matched files with the .gz extension to leave behind the logs

__move to qc_seq.ipynb__